In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
193,This movie is bad. I saw the rated and the unr...,negative
410,I'll make this brief. This was a joy to watch....,positive
820,and shot in Vancouver with the 'mountains' of ...,negative
477,THHE2 is entertaining in that you'll laugh a l...,negative
706,I am from the Dallas/Fort Worth area and lived...,negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
193,movie bad saw rated unrated version terrible n...,negative
410,i ll make brief joy watch may may effective ch...,positive
820,shot vancouver mountain low country south caro...,negative
477,thhe entertaining laugh lot cringe probably sa...,negative
706,dallas fort worth area lived arlington year mo...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    273
positive    227
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
193,movie bad saw rated unrated version terrible n...,0
410,i ll make brief joy watch may may effective ch...,1
820,shot vancouver mountain low country south caro...,0
477,thhe entertaining laugh lot cringe probably sa...,0
706,dallas fort worth area lived arlington year mo...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10759 stored elements and shape (500, 100)>

In [14]:
print(X[0])



<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 18 stored elements and shape (1, 100)>
  Coords	Values
  (0, 54)	2
  (0, 9)	3
  (0, 41)	1
  (0, 35)	3
  (0, 30)	2
  (0, 3)	1
  (0, 47)	1
  (0, 46)	2
  (0, 36)	1
  (0, 95)	1
  (0, 93)	1
  (0, 55)	1
  (0, 56)	1
  (0, 91)	1
  (0, 29)	1
  (0, 69)	1
  (0, 50)	1
  (0, 67)	1


In [15]:
X.tocoo().row

array([  0,   0,   0, ..., 499, 499, 499], shape=(10759,), dtype=int32)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/uveshmevawala/YT-Test-IMDB.mlflow')
dagshub.init(repo_owner='uveshmevawala', repo_name='YT-Test-IMDB', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as uveshmevawala

Initialized MLflow to track repo "uveshmevawala/YT-Test-IMDB"

Repository uveshmevawala/YT-Test-IMDB initialized!

2025/03/20 11:35:33 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c947a1246d2a466d88f7197df5d1f852', creation_time=1742470533032, experiment_id='0', last_update_time=1742470533032, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-20 11:47:27,985 - INFO - Starting MLflow run...
2025-03-20 11:47:28,285 - INFO - Logging preprocessing parameters...
2025-03-20 11:47:29,103 - INFO - Initializing Logistic Regression model...
2025-03-20 11:47:29,104 - INFO - Fitting the model...
2025-03-20 11:47:29,162 - INFO - Model training complete.
2025-03-20 11:47:29,163 - INFO - Logging model parameters...
2025-03-20 11:47:29,410 - INFO - Making predictions...
2025-03-20 11:47:29,413 - INFO - Calculating evaluation metrics...
2025-03-20 11:47:29,444 - INFO - Logging evaluation metrics...
2025-03-20 11:47:30,422 - INFO - Saving and logging the model...
2025/03/20 11:47:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 11:47:43,339 - INFO - Model training and logging completed in 15.05 seconds.
2025-03-20 11:47:43,343 - INFO - Executing Jupyter Notebook. This may take a while...
2025-03

🏃 View run judicious-doe-120 at: https://dagshub.com/uveshmevawala/YT-Test-IMDB.mlflow/#/experiments/0/runs/4c01bf48c71443d085f55dbd22554ab6
🧪 View experiment at: https://dagshub.com/uveshmevawala/YT-Test-IMDB.mlflow/#/experiments/0
